Import utilities 

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

Import Data

In [4]:
from Ingestion.kivadataloader import KivaDataLoader
import pandas as pd

m = KivaDataLoader()
cleaneduploans = m.get_clean_dataframe()

Connected to kivateam.cnd5idfvxmld.us-east-1.rds.amazonaws.com:5432/kiva
The process takes about 5 minutes to run.


Add Interactions/two-way

In [5]:
def add_interactions(cleaneduploans):
    # Get feature names
    combos = list(combinations(list(cleaneduploans.columns),2))
    colnames = list(cleaneduploans.columns) + ['_'.join(x) for x in combos]
    
    # Find Interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    cleaneduploans = poly.fit_transform(cleaneduploans)
    cleaneduploans = pd.DataFrame(cleaneduploans)
    cleaneduploans.columns = colnames
    
    # Remove interaction terms with all 0 values
    noint_indices = [i for i, x in enumerate(list((cleaneduploans == 0).all())) if x]
    cleaneduploans = cleaneduploans.drop(cleaneduploans.columns[noint_indices], axis=1)
    
    return cleaneduploans
    

In [6]:
# From Yamil - our columns excluding dates
X = cleaneduploans.drop(['posted_to_raised_days', 'posted_to_raised_bins', 'disburse_to_posted_days','disbursed_to_raised_days','partner_id'], axis=1)

In [7]:
# From Yamil - Using numeric values only
X = X.select_dtypes(exclude=['object','datetime64[ns, UTC]'])

In [8]:
# From Yamil - dropping NaN values
X.dropna(axis=1, inplace=True)

In [9]:
X.shape

(1170548, 39)

Don't run code below

In [ ]:
# Two-way interaction our features. As it currently stands, the 39 features will crearte C(39,2) = 741 new features.
# We should not run this code until we decide on the features. 
# We will utilize this if some of our original features produce weak relationship with target.
X = add_interactions(X)
print(X.head(5))